<a href="https://colab.research.google.com/github/skymined/study_data_analytics/blob/main/docs/pandas/CDA_RentalCarOfContractType_mixed_normal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 기본 세팅

In [1]:
# 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 코렙의 경우 설치 후 runtime  > restart 해야함

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 32 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 0s (50.5 MB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 121747 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
df_RCOCT = pd.read_csv('LetalCarOfContractType.csv')
df_RCOCT[:2]

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0


## 연속형이 정규 분포이며, 등분산일때


#### 예제 가설 : channel(항목 2), Mileage[:3000](정규분포)
- 분석 내용 : channel 중 서비스 방문과 홈쇼핑에 따른 마일리지 평균 차이가 있는가?
 + 귀무 가설 : 평균 차이가 없다.
 + 대립 가설 : 평균 차이가 있다.

### 등분산 검증

In [5]:
channel_list = df_RCOCT['channel'].value_counts().iloc[:2].index.to_list()
channel_list

['서비스 방문', '홈쇼핑/방송']

In [6]:
series_list = list()
for channel in channel_list:
  condition = f"channel == '{channel}'"
  series_amount = df_RCOCT.query(condition)['Mileage'].dropna() # condition에 따른 amount series 작성
  series_list.append(series_amount)
series_list[1].iloc[:5]

2     2363.0
6     2348.0
10    1555.0
11    1503.0
24    2947.0
Name: Mileage, dtype: float64

In [7]:
# 등분산 검증
import scipy.stats as stats
stats.levene(series_list[0], series_list[1])
# p>0.5이므로 정규 -> 등분산이다.

LeveneResult(statistic=2.3167489237265024, pvalue=0.12800023653017123)

In [8]:
# T test
stats.ttest_ind(series_list[0], series_list[1], equal_var=True) # 등분일 때 사용

TtestResult(statistic=-0.011263398682285758, pvalue=0.9910133902039651, df=24396.0)

### ANOVA Test
- 연속형이 정규분포이면서 범주형에 따른 항목이 등분산, 항목 개수가 3개 이상

In [9]:
channel_list = df_RCOCT['channel'].value_counts().iloc[:2].index.to_list()
channel_list

['서비스 방문', '홈쇼핑/방송']

In [10]:
series_list = list()
for channel in channel_list:
  condition = f"channel == '{channel}'"
  series_amount = df_RCOCT.query(condition)['Mileage'].dropna() # condition에 따른 amount series 작성
  series_list.append(series_amount)
series_list[1].iloc[:5]

2     2363.0
6     2348.0
10    1555.0
11    1503.0
24    2947.0
Name: Mileage, dtype: float64

In [11]:
stats.f_oneway(*series_list)

F_onewayResult(statistic=0.00012686414987622016, pvalue=0.991013390243794)

## 연속형이 정규분포이면서 범주형에 따른 항목이 미분산, 항목 개수가 2개 일때


In [12]:
# T test
stats.ttest_ind(series_list[0], series_list[1], equal_var=False) # 등분일 때 사용
# 등분산이 아닌 걸 가지고 하지 않았으므로 어떤 function을 사용하는 지만 보여준 것

TtestResult(statistic=-0.01126664522149353, pvalue=0.9910108002150029, df=24347.465526059503)